In [62]:
import numpy as np
import pandas as pd
import random
import sklearn as skl
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier


In [2]:
def generic_population(N, randomseed=100):
# generates a generic population    
    random.seed(randomseed)
    df_generic = pd.DataFrame({"age": np.round(np.random.uniform(-1.73,1.73,N),1),
                  "bmi": np.round(np.random.normal(0, 1,N),1),  
                  "hyp": np.random.binomial(1,0.20, N),
                  "gender": np.random.binomial(1,0.5, N)})
    return(df_generic)

def outcome_from_prob(event_probability, randomseed=100):
    random.seed(randomseed)    
    return np.random.binomial(1, event_probability, len(event_probability))

In [3]:
def check_event_prob(event_probability):
    #check violations for probabilities>0.99 and >1
    if (np.mean(event_probability > 0.99) > 0.001):
            print("Simulated probability was > 0.99 (and capped at 0.99): n=", 
                  sum(event_probability > 1),",",
                  np.round(np.mean(event_probability>0.99)*100,4),"%")
            
    print("Simulated probability > 1 (and capped at 0.99):  n=", sum(event_probability>1), ",",
                  np.round(np.mean(event_probability > 1)*100,4), "%" )
    if (np.round(np.mean(event_probability > 1)*100,4)>0.01):print ("\nConsider simulating lower prevalence.")
    return None 
    
def check_params(N, randomseed, expected_prevalence, simulation_type):
    #check that inputs are of the right type
    c1 = isinstance(N,int)
    c2 = isinstance(randomseed,int)
    c3 = (isinstance(expected_prevalence, float)|isinstance(expected_prevalence, int))
    if (isinstance(expected_prevalence, float)|isinstance(expected_prevalence, int)): 
        if(((expected_prevalence>=1)|(expected_prevalence<0))&(expected_prevalence!=-100)): 
            c3= False
    c4 = isinstance(simulation_type,str)
    if(isinstance(simulation_type,str)): 
        if simulation_type.lower()[0] not in ["l","n","c"]: c4 = False
        
    if (c1&c2&c3&c4 == False): 
        c= [c1,c2,c3] 
        b = ["N", "randomseed", "expected_prevalence"]
        print ("Error with input type:", [b[i] for i in range(len(c)) if c[i]==False])
        error= True
    else: 
        error= False
    
    return error

In [89]:
def linear_eventprob(df, normalising_parameter=1, verbose = False):     
    # applies transformation for the OR based on age, bmi, hyp and gender in df 
    # and calculates respective event probabilities     
    
    ######################-Linear- #############################
    event_OR = (2*df.bmi + 1.5*df.age + 1*df.hyp + 0*df.gender)
    ############################################################
    
    event_probability = normalising_parameter * (1 /(1+np.exp(-1*event_OR)))
    if verbose: check_event_prob(event_probability)
    
    event_probability[event_probability>0.99]=0.99    
    
    return(event_probability)


In [90]:
def nonlinear_eventprob(df, normalising_parameter=1, verbose = False):     
    # applies transformation for the OR based on age, bmi, hyp and gender in df 
    # and calculates respective event probabilities     
    ###################=- Non Linear -#################################
    bmi_beta = np.array([2 if (np.abs(x)> 1.5) else 
                         1 if (np.abs(x)>1) else 
                         0 for x in df["bmi"]])
    
    age_beta = np.array([2 if (x >=1) else 
                         0 for x in df["age"]])
    
    event_OR = (0.5*df.age + 1*age_beta + bmi_beta + 1*df.hyp + 0*df.gender)
    ###################################################################  
    event_probability = normalising_parameter * (1 /(1+np.exp(-1*event_OR)))
    if verbose: check_event_prob(event_probability)
    event_probability[event_probability>0.99]=0.99    
    return(event_probability)

In [91]:
def crossterms_eventprob(df, normalising_parameter=1, verbose = False):     
    # applies transformation for the OR based on age, bmi, hyp and gender in df 
    # and calculates respective event probabilities     
  
    ########################- Cross-terms -##########################
    #BMI impact is 2 for very low and high levels, 1 for high/ low level, 0 for normal range
    bmi_beta = np.array([2 if (np.abs(x)> 1.5) else 
                         1 if (np.abs(x)>1) else 
                         0 for x in df["bmi"]])
    
    # hypertension x age interaction: age > 1 has additional risk only if there is also hyp=1,
    # linear effect of age is still there 
    hyp_age_beta = np.array([2 if ((df.loc[i,"age"]>=0.5) & (df.loc[i,"hyp"]==1)) else 
                             0 for i in range(df.shape[0])])
    
    event_OR = bmi_beta + 1*df.hyp + 1* hyp_age_beta + 0.5*df.age + 0*df.gender
    ##################################################################
    
    event_probability = normalising_parameter * (1 /(1+np.exp(-1*event_OR)))
    if verbose: check_event_prob(event_probability)
    event_probability[event_probability>0.99]=0.99    
    return(event_probability)

## Main function simulating data with binary outome and linear/nonlinear or cross-terms

In [129]:
def simulate_binary_data(N=1000, randomseed = 100, expected_prevalence = -100, simulation_type = "linear"):
    # N - population size
    # prevalence - overall rate of the outcome in the population
    # randomseed - random seed for population simulation, set before applying numpy.random functions
    # simulation_type = linear/ non-linear/ cross-terms
    
    #check errors in inputs
    if check_params(N, randomseed, expected_prevalence,simulation_type): return None    
    random.seed(randomseed)
    ### calculate normalisation parameter to get a desired prevalence:  
    if (expected_prevalence!=-100):
        # generate a large population 
        df_0 = generic_population(10000, randomseed)
        # generate the outcome and compute normalising_parameter = expected/realised prevalence: 
        if simulation_type.lower()[0]=="l": 
            outcome_0 = outcome_from_prob(linear_eventprob(df_0, 1),randomseed)
        elif simulation_type.lower()[0]=="n":
            outcome_0 = outcome_from_prob(nonlinear_eventprob(df_0, 1),randomseed)
        elif simulation_type.lower()[0]=="c":
            outcome_0 = outcome_from_prob(crossterms_eventprob(df_0, 1),randomseed)
        else: 
            return None
        normalising_parameter = expected_prevalence/np.mean(outcome_0)
        
    else: normalising_parameter=1
    
    ### simulate the data 
    df = generic_population(N, randomseed)
    if simulation_type.lower()[0]=="l": 
        print ("Data type: linear")
        event_probability = linear_eventprob(df, normalising_parameter=normalising_parameter, verbose= True)
    elif simulation_type.lower()[0]=="n":
        print ("Data type: non-linear")
        event_probability = nonlinear_eventprob(df, normalising_parameter=normalising_parameter, verbose= True)
    elif simulation_type.lower()[0]=="c":
        print ("Data type: cross-terms")
        event_probability = crossterms_eventprob(df, normalising_parameter=normalising_parameter, verbose= True)
    else: 
        return None
    df["outcome"] = outcome_from_prob(event_probability, randomseed)
    
    return(df)

### Simulating data

#### Functions for running logistic regression and getting impact factors per 1std sorted by abs values

In [93]:
from statsmodels.discrete.discrete_model import Logit as logit

def get_coef_table(lin_reg, df, predictors_to_use):
    
    ''' lin_reg is a fitted statsmodels regression model
    Return a dataframe containing coefficients, pvalues, and the confidence intervals
    '''
    err_series = lin_reg.params - lin_reg.conf_int()[0]
    coef_df = pd.DataFrame({'predictor': err_series.index.values[:],
                            'LR_coef': lin_reg.params.values[:],
                            'LR_coef_std': err_series.values[:],
                            'LR_p_val': lin_reg.pvalues.round(4).values[0:],
                           })
    coef_df = coef_df.loc[coef_df.predictor !="C", :]
    coef_df['predictor_std'] = df[predictors_to_use].std().values
    coef_df["abs_coef_per_std"] = np.abs(coef_df.LR_coef / coef_df.predictor_std)
    coef_df = coef_df.sort_values(by="abs_coef_per_std", ascending = False)
    coef_df["rank"] = range(1,coef_df.shape[0]+1)
    coef_df.loc[coef_df["LR_p_val"]>0.05, "noise?"] = 1
    coef_df.loc[coef_df["LR_p_val"]<=0.05, "noise?"] = 0
    return coef_df

def LR_impact_factors(df, predictors_to_use):
    # fits logistic regression and 
    # returns sorted coefficients normalized per 1 std 
    # of the underlying predictor move 
    
    y = df.outcome
    X = df[predictors_to_use]; X["C"]=1
    #fit logistic regression
    lr_model = logit(y, X).fit()
    # return coefficient table
    res = get_coef_table(lr_model, df, predictors_to_use)
    return res, lr_model.summary() 


In [121]:
def cross_validate_lr(classifier_model, df, predictors):
    X = df[predictors]
    y = df.outcome 
    my_scoring = ['f1_macro', 'roc_auc']
    scores = cross_validate(classifier_model, X, y, cv=10, scoring = my_scoring)
    res = pd.DataFrame(scores).describe().loc[["mean", "std"],['test_f1_macro', 'test_roc_auc'] ]
    return res


## 1) Linear data. Sumulation, description and logistic regression coefficients:

In [124]:
#simulation:
df_lin = simulate_binary_data(20000, simulation_type = "linear")
df_lin.describe().iloc[[1,2,3]]

Data type: linear
Simulated probability was > 0.99 (and capped at 0.99): n= 0 , 4.075 %
Simulated probability > 1 (and capped at 0.99):  n= 0 , 0.0 %


,age,bmi,hyp,gender,outcome
mean,-0.002910,-0.003690,0.197400,0.5035,0.526050
std,0.997472,0.996405,0.398047,0.5000,0.499333
min,-1.700000,-4.400000,0.000000,0.0000,0.000000


### Variable importance from coefficients in LR:

In [126]:
#coefficients:
predictors_linear = ["age", "bmi", "hyp", "gender"]
LR_impact_factors(df_lin,predictors_linear)[0]

Optimization terminated successfully.
         Current function value: 0.398013
         Iterations 7


,predictor,LR_coef,LR_coef_std,LR_p_val,predictor_std,abs_coef_per_std,rank,noise?
2,hyp,1.026414,0.101023,0.00,0.398047,2.578627,1,0.0
1,bmi,2.011715,0.060779,0.00,0.996405,2.018974,2,0.0
0,age,1.523275,0.049870,0.00,0.997472,1.527135,3,0.0
3,gender,-0.092036,0.077532,0.02,0.500000,0.184072,4,0.0


Reminder: 
    ######################-Linear- #############################
    event_OR = (2*df.bmi + 1.5*df.age + 1*df.hyp + 0*df.gender)
    ############################################################
So expect non-standardised coefficients to be 
* age = 1.5
* bmi = 2,
* hyp = 1,
* gender = 0

In [127]:
# cross-validated performance of different models
logit_model = LogisticRegression(random_state=102, solver = "lbfgs",
                                 penalty = "none", max_iter = 500)
nl1 = cross_validate_lr(logit_model, df_lin, predictors_linear)              
nl1["data_type"] = "linear"; nl1["model_type"] = "linear"
nl3 = cross_validate_lr(rf_model, df_lin, predictors_linear)
nl3["data_type"] = "linear"; nl3["model_type"] = "random_forest"
performance_l = nl1.append(nl3)
performance_l

,test_f1_macro,test_roc_auc,data_type,model_type
mean,0.814088,0.900048,linear,linear
std,0.011252,0.007610,linear,linear
mean,0.800435,0.887749,linear,random_forest
std,0.011127,0.008473,linear,random_forest


## 2) Non-linear data. Sumulation, description and logistic regression coefficients:

In [96]:
df_nonlin = simulate_binary_data(20000, randomseed = 42,  simulation_type = "non-linear")
df_nonlin.describe().iloc[[1,2,3]]

Data type: non-linear
Simulated probability was > 0.99 (and capped at 0.99): n= 0 , 2.395 %
Simulated probability > 1 (and capped at 0.99):  n= 0 , 0.0 %


,age,bmi,hyp,gender,outcome
mean,0.002395,-0.001395,0.199950,0.501200,0.665900
std,0.993117,0.995762,0.399972,0.500011,0.471687
min,-1.700000,-4.000000,0.000000,0.000000,0.000000


### Variable importance from coefficients in LR:
* a) using linear model
* b) using the "true" model which has all modelled linear and non-linear terms 

In [130]:
# a)  coefficients in a linear model ("linear" "variable importance")
predictors_linear= ["age", "bmi", "hyp", "gender"]
LR_impact_factors(df_nonlin, predictors_linear)[0]

Optimization terminated successfully.
         Current function value: 0.560915
         Iterations 6


/Users/diana/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,predictor,LR_coef,LR_coef_std,LR_p_val,predictor_std,abs_coef_per_std,rank,noise?
2,hyp,1.110786,0.129904,0.0000,0.397218,2.796414,1,0.0
0,age,0.828046,0.048686,0.0000,0.996054,0.831327,2,0.0
3,gender,-0.054949,0.089935,0.2311,0.500017,0.109893,3,1.0
1,bmi,-0.001567,0.045589,0.9463,0.985855,0.001590,4,1.0


**! bmi is insignificant in a linear model, although it has a high impact on the outcome**

In [131]:
# b) coefficients in the optimal non-linear model ("true" "variable importance")
# add coefficients to represent the non-linear terms as they were modelled: 
df_nonlin[ "bmi_cat_veryhigh"] = 0
df_nonlin.loc[ np.abs(df_nonlin.bmi>1.5), "bmi_cat_veryhigh"] = 1
df_nonlin["bmi_cat_high"] = 0
df_nonlin.loc[(np.abs(df_nonlin.bmi<=1.5))&(np.abs(df_nonlin.bmi>1)),
              "bmi_cat_high"] = 1
df_nonlin.loc[ np.abs(df_nonlin.age > 1), "age_cat"] = 1
df_nonlin.loc[ np.abs(df_nonlin.age<= 1), "age_cat"] = 0

In [105]:
predictors_nonlinear = ["age", "hyp", "gender", "age_cat", 
                        "bmi_cat_high", "bmi_cat_veryhigh"]

In [106]:
# impact factors in logistic regression which accounts 
# for the exact non-linear effects modelled: 
LR_impact_factors(df_nonlin, predictors_nonlinear)[0]

Optimization terminated successfully.
         Current function value: 0.533972
         Iterations 7


/Users/diana/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,predictor,LR_coef,LR_coef_std,LR_p_val,predictor_std,abs_coef_per_std,rank,noise?
5,bmi_cat_veryhigh,1.770804,0.274619,0.0000,0.232807,7.606313,1,0.0
3,age_cat,1.689723,0.238749,0.0000,0.394517,4.283017,2,0.0
4,bmi_cat_high,0.831784,0.178801,0.0000,0.278598,2.985610,3,0.0
1,hyp,1.140067,0.130643,0.0000,0.397218,2.870130,4,0.0
0,age,0.580300,0.059782,0.0000,0.996054,0.582599,5,0.0
2,gender,-0.046551,0.091903,0.3208,0.500017,0.093099,6,1.0


    ###################=- Non Linear -#################################
    bmi_beta = np.array([2 if (np.abs(x)> 1.5) else 
                         1 if (np.abs(x)>1) else 
                         0 for x in df["bmi"]])
    age_beta = np.array([2 if (x >=1) else 
                         0 for x in df["age"]])
    event_OR = (0.5*df.age + 1*age_beta + bmi_beta + 1*df.hyp + 0*df.gender)
    ################################################################### 
So we expect 
* beta bmi_cat_veryhigh = 2; bmi_cat_high=1
* age_cat = 2;    age = 0.5;    hyp = 1;    gender =0

**! bmi_cat_high for those with the bmi above 1std deviation is most important now**

In [120]:
# cross-validated performance of different models
logit_model = LogisticRegression(random_state=102, solver = "lbfgs",
                                 penalty = "none", max_iter = 500)
nl1 = cross_validate_lr(logit_model, df_nonlin, predictors_linear)              
nl1["data_type"] = "non_linear"; nl1["model_type"] = "linear"
nl2 = cross_validate_lr(logit_model, df_nonlin, predictors_nonlinear)              
nl2["data_type"] = "non_linear"; nl2["model_type"] = "best_linear"
nl3 = cross_validate_lr(rf_model, df_nonlin, predictors_linear)
nl3["data_type"] = "non_linear"; nl3["model_type"] = "random_forest"
performance_nl = nl1.append(nl2).append(nl3)
performance_nl

,test_f1_macro,test_roc_auc,data_type,model_type
mean,0.628338,0.735693,non_linear,linear
std,0.025247,0.017371,non_linear,linear
mean,0.653384,0.762706,non_linear,best_linear
std,0.019945,0.014370,non_linear,best_linear
mean,0.698450,0.786342,non_linear,random_forest
std,0.011416,0.014236,non_linear,random_forest


## 3) Cross-terms data. Sumulation, description and logistic regression coefficients:

In [108]:
df_crossterms = simulate_binary_data(20000, randomseed=54, simulation_type = "cross-terms")
df_crossterms.describe().iloc[[1,2,3]]

Data type: cross-terms
Simulated probability was > 0.99 (and capped at 0.99): n= 0 , 1.545 %
Simulated probability > 1 (and capped at 0.99):  n= 0 , 0.0 %


,age,bmi,hyp,gender,outcome
mean,0.007425,0.009320,0.20020,0.499450,0.636900
std,1.001979,0.993646,0.40016,0.500012,0.480905
min,-1.700000,-3.900000,0.00000,0.000000,0.000000


### Variable importance from coefficients in LR:

In [128]:
# linear model, impact factors: 
predictors_linear= ["age", "bmi", "hyp", "gender"]
LR_impact_factors(df_crossterms,predictors_linear)[0]

Optimization terminated successfully.
         Current function value: 0.604167
         Iterations 6


/Users/diana/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,predictor,LR_coef,LR_coef_std,LR_p_val,predictor_std,abs_coef_per_std,rank,noise?
2,hyp,1.341097,0.091441,0.000,0.400160,3.351402,1,0.0
0,age,0.508983,0.031126,0.000,1.001979,0.507978,2,0.0
3,gender,0.032471,0.060641,0.294,0.500012,0.064940,3,1.0
1,bmi,0.021393,0.030556,0.170,0.993646,0.021529,4,1.0


! again, bmi lost its importance in a linear model

In [115]:
# coefficients in the optimal non-linear model ("true" "variable importance")
# add coefficients to represent the non-linear terms as those were modelled: 
df_crossterms.loc[ np.abs(df_crossterms.bmi>1.5), "bmi_cat_veryhigh"] = 1
df_crossterms.loc[ np.abs(df_crossterms.bmi<=1.5), "bmi_cat_veryhigh"] = 0
df_crossterms.loc[(np.abs(df_crossterms.bmi<=1.5))&(np.abs(df_crossterms.bmi>1)),
                  "bmi_cat_high"] = 1
df_crossterms.loc[ np.abs(df_crossterms.bmi<=1) | np.abs(df_crossterms.bmi>1.5),
                  "bmi_cat_high"] = 0

df_crossterms["age_x_hyp"] = 0
df_crossterms.loc[ np.abs(df_crossterms.age>0.5) & df_crossterms.hyp==1,
                  "age_x_hyp"] = 1
predictors_crossterms = ["age", "hyp", "gender", "age_x_hyp",  
                         "bmi_cat_high", "bmi_cat_veryhigh"]

In [116]:
LR_impact_factors(df_crossterms,predictors_crossterms)[0]

Optimization terminated successfully.
         Current function value: 0.583295
         Iterations 8


/Users/diana/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,predictor,LR_coef,LR_coef_std,LR_p_val,predictor_std,abs_coef_per_std,rank,noise?
3,age_x_hyp,2.129241,0.426268,0.0000,0.253460,8.400682,1,0.0
5,bmi_cat_veryhigh,1.892552,0.191909,0.0000,0.239428,7.904472,2,0.0
1,hyp,1.103656,0.098961,0.0000,0.400160,2.758038,3,0.0
4,bmi_cat_high,0.693679,0.117209,0.0000,0.280740,2.470897,4,0.0
0,age,0.474563,0.032557,0.0000,1.001979,0.473626,5,0.0
2,gender,0.027881,0.061613,0.3751,0.500012,0.055760,6,1.0


    ########################- Cross-terms -##########################
    bmi_beta = np.array([2 if (np.abs(x)> 1.5) else 
                         1 if (np.abs(x)>1) else 
                         0 for x in df["bmi"]])
    # hypertension x age interaction: age > 0.5 
    # has additional risk only if there is also hyp=1,
    # linear effect of age is still there 
    hyp_age_beta = np.array([2 if ((df.loc[i,"age"]>=0.5) & (df.loc[i,"hyp"]==1)) else 
                             0 for i in range(df.shape[0])])
    event_OR = bmi_beta + 1*df.hyp + 1* hyp_age_beta + 0.5*df.age + 0*df.gender
    ##################################################################
So, LR coefficients should be 
 * age = 0.5, 
 * Binary(age > 0.5 & hyp =1) = 2, 
 * gender = 0, 
 * Binary(abs(bmi)>1.5))=2,  
 * Binary(1<abs(bmi)<1.5))=1

In [119]:
# cross-validated performance of different models
logit_model = LogisticRegression(random_state=102, solver = "lbfgs",
                                 penalty = "none", max_iter = 500)
nl1 = cross_validate_lr(logit_model, df_crossterms, predictors_linear)              
nl1["data_type"] = "cross_terms"; nl1["model_type"] = "linear"
nl2 = cross_validate_lr(logit_model, df_crossterms, predictors_crossterms)              
nl2["data_type"] = "cross_terms"; nl2["model_type"] = "best_linear"
nl3 = cross_validate_lr(rf_model, df_crossterms, predictors_linear)
nl3["data_type"] = "cross_terms"; nl3["model_type"] = "random_forest"
performance_ct = nl1.append(nl2).append(nl3)
performance_ct

,test_f1_macro,test_roc_auc,data_type,model_type
mean,0.589591,0.678297,cross_terms,linear
std,0.009617,0.010169,cross_terms,linear
mean,0.609648,0.706952,cross_terms,best_linear
std,0.012512,0.013291,cross_terms,best_linear
mean,0.638089,0.730613,cross_terms,random_forest
std,0.014881,0.013898,cross_terms,random_forest


### Results for the internal cross-validation

In [132]:
performance_l.append([performance_nl, performance_ct], sort=False).T.iloc[:,0:20:2]

,mean,mean,mean,mean,mean,mean,mean,mean
test_f1_macro,0.814088,0.800435,0.628338,0.653384,0.69845,0.589591,0.609648,0.638089
test_roc_auc,0.900048,0.887749,0.735693,0.762706,0.786342,0.678297,0.706952,0.730613
data_type,linear,linear,non_linear,non_linear,non_linear,cross_terms,cross_terms,cross_terms
model_type,linear,random_forest,linear,best_linear,random_forest,linear,best_linear,random_forest


In [133]:
performance_l.append([performance_nl, performance_ct], sort=False).T.iloc[:,1:20:2]

,std,std,std,std,std,std,std,std
test_f1_macro,0.0112522,0.0111266,0.0252468,0.0199452,0.0114158,0.00961735,0.0125124,0.014881
test_roc_auc,0.0076097,0.00847265,0.0173712,0.0143701,0.0142359,0.0101695,0.0132912,0.013898
data_type,linear,linear,non_linear,non_linear,non_linear,cross_terms,cross_terms,cross_terms
model_type,linear,random_forest,linear,best_linear,random_forest,linear,best_linear,random_forest
